# **Контекстные менеджеры в Python: теоретические основы**

In [ ]:
## **1. Определение и назначение**
# Контекстный менеджер (context manager) — это объект, который определяет контекст выполнения блока кода, обеспечивая:  
# - **Инициализацию ресурсов** при входе в контекст.  
# - **Корректное освобождение** ресурсов при выходе (даже в случае исключения).  

# Основная цель — замена ручного управления ресурсами (например, закрытие файлов, соединений) на автоматическое.

# ---

In [1]:
## **2. Протокол контекстного менеджера**
# Любой объект может стать контекстным менеджером, если реализует **протокол `__enter__`/`__exit__`**:

### **Методы:**
# 1. **`__enter__(self)`**  
#    - Вызывается при входе в блок `with`.  
#    - Возвращаемое значение передаётся в переменную после `as`.  

# 2. **`__exit__(self, exc_type, exc_val, exc_tb)`**  
#    - Вызывается при выходе из блока `with`.  
#    - Параметры (`exc_type`, `exc_val`, `exc_tb`):  
#      - Если исключения не было — все `None`.  
#      - Если было — содержат тип, значение и traceback исключения.  
#    - Если метод возвращает `True`, исключение подавляется.  

### **Пример реализации:**
# ```python
class MyContextManager:
    def __enter__(self):
        print("Вход в контекст")
        return self  # Объект для as

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("Выход из контекста")
        if exc_type:
            print(f"Исключение: {exc_val}")
        return True  # Подавить исключение
# ```

# ---

In [2]:
## **3. Синтаксис `with`**
# Конструкция `with` гарантирует вызов `__exit__` при любом выходе из блока (нормальном или через исключение):
# ```python
with MyContextManager() as cm:
    print("Внутри контекста")
    # cm — объект, возвращённый __enter__
# ```

# ---

Вход в контекст
Внутри контекста
Выход из контекста


In [3]:
## **4. Теоретические аспекты**

### **a. Безопасность ресурсов**
# Контекстные менеджеры решают проблему **утечки ресурсов** (файлы, сокеты, БД), так как `__exit__` вызывается **всегда**, даже при:  
# - Возврате (`return`).  
# - Исключении.  
# - Выходе из функции.  

### **b. Иерархия контекстов**
# Несколько менеджеров можно комбинировать:  
# ```python
# with open("a.txt") as f1, open("b.txt") as f2:
    # Оба файла гарантированно закроются
# ```

### **c. Подавление исключений**
# Если `__exit__` возвращает `True`, исключение не распространяется дальше.  
# Пример подавления `ValueError`:  
# ```python
class SuppressValueError:
    def __exit__(self, exc_type, exc_val, exc_tb):
        return exc_type is ValueError  # Подавить только ValueError
# ```

# ---

In [4]:
## **5. Встроенные контекстные менеджеры**
# Python включает готовые менеджеры для частых задач:

# | Менеджер                     | Ресурс                 |
# |------------------------------|------------------------|
# | `open(file)`                 | Файлы                  |
# | `threading.Lock()`           | Блокировки потоков     |
# | `contextlib.redirect_stdout` | Перенаправление вывода |
# | `sqlite3.connect(db)`        | Соединения с БД        |

# ---

In [5]:
## **6. Генераторы как контекстные менеджеры**
# Модуль `contextlib` позволяет создавать менеджеры из генераторов через `@contextmanager`:  
# ```python
from contextlib import contextmanager

@contextmanager
def timer():
    start = time.time()
    try:
        yield start  # Аналог __enter__
    finally:
        print(f"Time: {time.time() - start}s")  # Аналог __exit__
# ```

# **Принцип работы**:  
# 1. Код до `yield` — инициализация (аналог `__enter__`).  
# 2. `yield` возвращает объект для `as`.  
# 3. Код после `yield` — очистка (аналог `__exit__`).  

# ---

In [7]:
## **7. Математическая модель**
# Контекстные менеджеры можно описать как **функции высшего порядка**, гарантирующие выполнение условий:  
# ```
with Resource() as r:
    f(r)
# ```
# Эквивалентно:  
# ```
Resource().__enter__() → r  
try:  
    f(r)  
finally:  
    Resource().__exit__()  
# ```

# ---

NameError: name 'Resource' is not defined

In [ ]:
## **Заключение**
# Контекстные менеджеры — это реализация **идиомы RAII** (Resource Acquisition Is Initialization) в Python:  
# - **Инкапсулируют** логику управления ресурсами.  
# - **Гарантируют** безопасность при исключениях.  
# - **Упрощают** код, устраняя шаблонные `try/finally`.  

# Их использование делает код **надежнее** и **чище**, особенно при работе с внешними ресурсами.

In [8]:
## **8. Ограничения**
# 1. **Нельзя войти в контекст дважды** без повторного вызова `with`.  
# 2. **Состояние объекта** должно быть консистентным после `__exit__`.  
# 3. **Вложенность** контекстов может усложнить отладку.  

# ---

# **Практические примеры использования контекстных менеджеров в Python**

In [9]:
# Контекстные менеджеры (`with`) в Python обеспечивают безопасное управление ресурсами. Вот **все основные практические применения** с примерами:

# ---

In [10]:
## **1. Работа с файлами**
# Автоматическое закрытие файлов даже при возникновении исключений.

# ```python
# Чтение файла
with open('data.txt', 'r') as file:
    content = file.read()
# Файл закрыт автоматически

# Запись в файл
with open('output.txt', 'w') as file:
    file.write('Hello, World!')
# ```

# ---

FileNotFoundError: [Errno 2] No such file or directory: 'data.txt'

In [ ]:
## **2. Работа с базами данных**
# Автоматическое закрытие соединений и транзакций.

### SQLite
# ```python
import sqlite3

with sqlite3.connect('database.db') as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM users')
    data = cursor.fetchall()
# Соединение закрыто
# ```

### PostgreSQL (psycopg2)
# ```python
import psycopg2

with psycopg2.connect(dbname='test', user='postgres') as conn:
    with conn.cursor() as cursor:  # Вложенный контекст
        cursor.execute('SELECT version()')
# Соединение и курсор закрыты
# ```

# ---

In [ ]:
## **3. Многопоточность: блокировки**
# Автоматическое освобождение блокировок.

# ```python
import threading

lock = threading.Lock()

with lock:  # Захватываем блокировку
    print('Критическая секция')
# Блокировка освобождена
# ```

# ---

In [ ]:
## **4. Временные директории**
# Автоматическое удаление временных файлов.

# ```python
import tempfile

with tempfile.TemporaryDirectory() as tmp_dir:
    print(f'Временная директория: {tmp_dir}')
    # Файлы в tmp_dir будут удалены автоматически
# ```

# ---

In [ ]:
## **5. Таймеры выполнения**
# Замер времени выполнения блока кода.

# ```python
from contextlib import contextmanager
import time

@contextmanager
def timer():
    start = time.time()
    yield
    print(f'Время выполнения: {time.time() - start:.2f} сек')

with timer():
    # Долгая операция
    time.sleep(1)
# ```

# ---

In [ ]:
## **6. Перенаправление вывода**
# Временное перенаправление `stdout`/`stderr`.

# ```python
from contextlib import redirect_stdout
import io

buffer = io.StringIO()

with redirect_stdout(buffer):  # Перенаправляем вывод в buffer
    print('Это не выведется в консоль')

print(f'Захваченный вывод: {buffer.getvalue()}')
# ```

# ---

In [ ]:
## **7. Изменение рабочей директории**
# Временное изменение текущей директории.

# ```python
from contextlib import contextmanager
import os

@contextmanager
def change_dir(path):
    old_dir = os.getcwd()
    os.chdir(path)
    yield
    os.chdir(old_dir)

with change_dir('/tmp'):
    print(f'Текущая директория: {os.getcwd()}')
# Автоматически вернулись в исходную директорию
# ```

# ---

In [ ]:
## **8. HTTP-сессии (requests)**
# Автоматическое закрытие сессии.

# ```python
import requests

with requests.Session() as session:
    response = session.get('https://api.example.com/data')
    print(response.json())
# Сессия закрыта
# ```

# ---

In [ ]:
## **9. Работа с matplotlib**
# Автоматическое закрытие фигур.

# ```python
import matplotlib.pyplot as plt

with plt.figure() as fig:
    plt.plot([1, 2, 3])
    plt.savefig('plot.png')
# Ресурсы освобождены
# ```

# ---

In [ ]:
## **10. Подавление исключений**
# Временное игнорирование ошибок.

# ```python
from contextlib import suppress

with suppress(FileNotFoundError):  # Игнорируем FileNotFoundError
    os.remove('non_existent_file.txt')
print('Программа продолжает работу')
# ```

# ---

In [ ]:
## **11. Сетевое программирование (сокеты)**
# Автоматическое закрытие сокетов.

# ```python
import socket

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(('example.com', 80))
    s.sendall(b'GET / HTTP/1.1\r\nHost: example.com\r\n\r\n')
    data = s.recv(1024)
# Сокет закрыт
# ```

# ---

In [ ]:
## **12. Работа с zip-архивами**
# Автоматическое закрытие архивов.

# ```python
import zipfile

with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    zip_ref.extractall('extracted_files')
# Архив закрыт
# ```

# ---

In [ ]:
## **13. Пользовательские контекстные менеджеры**
# Создание собственных менеджеров.

# ```python
class DatabaseConnection:
    def __enter__(self):
        print('Подключение к БД')
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('Отключение от БД')
        if exc_type:
            print(f'Ошибка: {exc_val}')
        return True  # Подавить исключение

with DatabaseConnection() as db:
    print('Выполняем запросы')
    # raise ValueError('Ошибка запроса')  # Будет подавлено
# ```

# ---

In [11]:
## **14. Асинхронные контекстные менеджеры**
# Для работы с `async/await`.

# ```python
import aiohttp
import asyncio

async def fetch_data():
    async with aiohttp.ClientSession() as session:  # Асинхронный with
        async with session.get('https://api.example.com/data') as resp:
            return await resp.json()

data = asyncio.run(fetch_data())
# ```

# ---

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
## **15. Тестирование: мокирование**
# Временная замена объектов при тестировании.

# ```python
from unittest.mock import patch

with patch('module.function', return_value=42):
    result = module.function()  # Вернет 42
# Оригинальная function восстановлена
# ```

# ---

In [ ]:
## **Итог**
# Контекстные менеджеры применяются для:
# 1. Управления **ресурсами** (файлы, сетевые соединения, БД).
# 2. Обеспечения **безопасности** при исключениях.
# 3. Временных **изменений состояния** (директории, настройки).
# 4. **Автоматизации** повторяющихся операций (таймеры, логирование).

# Их использование делает код **чище** и **надежнее**, избавляя от ручного управления жизненным циклом ресурсов.